In [1]:
import argparse
import glob

import seaborn as sns
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import datasets, plotting
from statsmodels.stats.multitest import multipletests
from src import custom_plotting as cp
from nilearn import surface
from pathlib import Path

from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
def load_noise_ceiling(mask_dir, sid, mask):
    noise_ceiling = np.load(f'{mask_dir}/sub-{sid}_set-test_stat-rho_statmap.npy')
    noise_ceiling = noise_ceiling[mask]
    noise_ceiling[noise_ceiling < 0.] = np.nan
    return noise_ceiling

In [ ]:
def filter_r(rs, ps, p_crit=0.05, correct=True, threshold=True):
    if correct:
        _, ps_corrected, _, _ = multipletests(ps, method='fdr_bh')
    else:
        ps_corrected = ps.copy()

    if threshold:
        rs[ps_corrected >= p_crit] = 0.
    else:
        rs[rs < 0.] = 0.
    return rs, ps_corrected

In [ ]:
sid = '01'
data_dir='/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/raw'
out_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim'
mask_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim/Reliability'
figure_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/reports/figures/PlotVoxelEncodingTest'

fsaverage = datasets.fetch_surf_fsaverage()
cmap = sns.color_palette('magma', as_cmap=True)
mask_im = nib.load(f'{mask_dir}/sub-{sub}_set-test_stat-rho_statmap.nii.gz')
roi_parcel = None

In [ ]:
r_true = np.load(f'{out_dir}/VoxelPermutationTest/sub-{sid}_{model}_rs.npy')
p = np.load(f'{out_dir}/VoxelPermutationTest/sub-{sid}_{model}_ps.npy')
mask = np.load(f'{out_dir}/VoxelPermutationTest/sub-{sid}_mask.npy')

In [5]:
r_true, p_corrected = filter_r(r_true, p)
noise_ceiling = load_noise_ceiling(mask_dir, sid, mask)
r_normed = np.divide(r_true, noise_ceiling)
volume = mkNifti(r_normed, mask, mask_im)
texture = {'left': surface.vol_to_surf(volume, fsaverage['pial_left']),
           'right': surface.vol_to_surf(volume, fsaverage['pial_right'])}

In [ ]:
hemi = 'right'
plotting.view_surf(fsaverage[f'infl_{hemi}'],
                   texture[hemi],
                   bg_map=fsaverage[f'sulc_{hemi}'],
                   symmetric_cmap=False,
                   threshold=0.01, vmax=1,
                   cmap=cmap)

In [ ]:
plot_surface_stats(fsaverage, texture,
                   roi=roi_parcel,
                   cmap=cmap,
                   modes=['lateral', 'ventral'],
                   threshold=0.01,
                   vmax=1.,
                   output_file=f'{figure_dir}/test.png')